In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pickle
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
import copy
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
warnings.filterwarnings(action='ignore')
from epftoolbox.evaluation import MAE, sMAPE

# DNN

In [2]:
df = pd.read_csv(r"Data\EPFToolbox\BE.csv")

In [3]:
df

,Date,Prices,Generation forecast,System load forecast
0,2011-01-09 00:00:00,32.54,63065.0,63000.0
1,2011-01-09 01:00:00,21.55,62715.0,58800.0
2,2011-01-09 02:00:00,15.71,61952.0,58500.0
3,2011-01-09 03:00:00,10.58,59262.0,54300.0
4,2011-01-09 04:00:00,10.32,56883.0,51900.0
...,...,...,...,...
52411,2016-12-31 19:00:00,40.84,70329.0,73957.0
52412,2016-12-31 20:00:00,40.10,69121.0,72544.0
52413,2016-12-31 21:00:00,36.00,66647.0,69451.0
52414,2016-12-31 22:00:00,35.00,65886.0,67823.0


In [4]:
df['Date'] = pd.to_datetime(df.Date)

In [5]:
df.index = df.Date

In [6]:
df = df.drop('Date', axis = 1)

In [7]:
df.columns = ['Price', 'Exogenous 1', 'Exogenous 2']

In [8]:
df_train = df[df.index < dt.datetime(2015, 1, 1)]
df_test = df[df.index >= dt.datetime(2015, 1, 1)]

In [9]:
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

In [10]:
forecast_dates = forecast.index

In [11]:
from epftoolbox.models import hyperparameter_optimizer

In [12]:
df.to_csv("Data/EPFToolbox/BE_dnn.csv")

In [13]:
# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = "BE_dnn"

# Number of years (a year is 364 days) in the test dataset.
years_test = 2

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2015 00:00"
end_test_date = "31/12/2016 23:00"

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = 4

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 1

# Number of iterations for hyperparameter optimization
max_evals = 100

path_datasets_folder = "./Data/EPFToolbox/"
path_hyperparameters_folder = "./experimental_files/"

# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00


Tested 1/100 iterations.
Best MAE - Validation Dataset
  MAE: 11.0 | sMAPE: 23.82 %

Best MAE - Test Dataset
  MAE: 14.1 | sMAPE: 33.68 %


Tested 2/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.6 | sMAPE: 14.57 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.73 %


Tested 3/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.6 | sMAPE: 14.57 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.73 %


Tested 4/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.6 | sMAPE: 14.57 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.73 %


Tested 5/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.6 | sMAPE: 14.57 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.73 %


Tested 6/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.6 | sMAPE: 14.57 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.73 %


Tested 7/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.6 | sMAPE: 14.57 %

Best MAE - Test Dataset
  M



Tested 60/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.8 | sMAPE: 12.95 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.46 %


Tested 61/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.8 | sMAPE: 12.95 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.46 %


Tested 62/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.8 | sMAPE: 12.95 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.46 %


Tested 63/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.8 | sMAPE: 12.95 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.46 %


Tested 64/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.8 | sMAPE: 12.95 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.46 %


Tested 65/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.8 | sMAPE: 12.95 %

Best MAE - Test Dataset
  MAE: 8.6 | sMAPE: 20.46 %


Tested 66/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.8 | sMAPE: 12.94 %

Best MAE - Test Dataset
  MAE: 8.1 | sMAPE: 19.45 %


Tested 67/100 iterations.

In [14]:
"""
Example for using the DNN model for forecasting prices with daily recalibration
"""

# Author: Jesus Lago

# License: AGPL-3.0 License

import pandas as pd
import numpy as np
import argparse
import os

from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE
from epftoolbox.models import DNN


path_datasets_folder = "./Data/EPFToolbox/"
path_hyperparameters_folder = "./experimental_files/"
path_recalibration_folder = os.path.join('.', 'Resultados')
path_hyperparameter_folder = os.path.join('.', 'experimental_files')

dataset = "BE_dnn"

# Number of years (a year is 364 days) in the test dataset.
years_test = 2

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2015 00:00"
end_test_date = "31/12/2016 23:00"

# Defining train and testing data
df_train, df_test = read_data(dataset=dataset, years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

# Defining unique name to save the forecast
forecast_file_name = 'dnn_BE.csv'

forecast_file_path = os.path.join(path_recalibration_folder, forecast_file_name)

# # Defining empty forecast array and the real values to be predicted in a more friendly format
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

# # If we are not starting a new recalibration but re-starting an old one, we import the
# # existing files and print metrics 
# if not new_recalibration:
#     # Import existinf forecasting file
#     forecast = pd.read_csv(forecast_file_path, index_col=0)
#     forecast.index = pd.to_datetime(forecast.index)

#     # Reading dates to still be forecasted by checking NaN values
#     forecast_dates = forecast[forecast.isna().any(axis=1)].index

#     # If all the dates to be forecasted have already been forecast, we print information
#     # and exit the script
#     if len(forecast_dates) == 0:

#         mae = np.mean(MAE(forecast.values.squeeze(), real_values.values))
#         smape = np.mean(sMAPE(forecast.values.squeeze(), real_values.values)) * 100
#         print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format('Final metrics', smape, mae))
    
# else:
#     forecast_dates = forecast.index

forecast_dates = forecast.index

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 4
experiment_id = 1

model = DNN(
    experiment_id=experiment_id, path_hyperparameter_folder=path_hyperparameter_folder, nlayers=nlayers, 
    dataset=dataset, years_test=years_test, shuffle_train=shuffle_train, data_augmentation=data_augmentation,
    calibration_window=calibration_window)


# For loop over the recalibration dates
for date in forecast_dates:

    # For simulation purposes, we assume that the available data is
    # the data up to current date where the prices of current date are not known
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=23), :]], axis=0)

    # We extract real prices for current date and set them to NaN in the dataframe of available data
    data_available.loc[date:date + pd.Timedelta(hours=23), 'Price'] = np.NaN

    # Recalibrating the model with the most up-to-date available data and making a prediction
    # for the next day
    Yp = model.recalibrate_and_forecast_next_day(df=data_available, next_day_date=date)

    # Saving the current prediction
    forecast.loc[date, :] = Yp

    # Computing metrics up-to-current-date
    mae = np.mean(MAE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) 
    smape = np.mean(sMAPE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) * 100

    # Pringint information
    print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(str(date)[:10], smape, mae))

#     # Saving forecast
#     forecast.to_csv(forecast_file_path)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00
2015-01-01 - sMAPE: 10.92%  |  MAE: 4.420
2015-01-02 - sMAPE: 10.94%  |  MAE: 4.305
2015-01-03 - sMAPE: 10.64%  |  MAE: 4.191
2015-01-04 - sMAPE: 9.70%  |  MAE: 3.776
2015-01-05 - sMAPE: 10.01%  |  MAE: 4.138
2015-01-06 - sMAPE: 9.90%  |  MAE: 4.206
2015-01-07 - sMAPE: 9.18%  |  MAE: 3.934
2015-01-08 - sMAPE: 9.13%  |  MAE: 3.920
2015-01-09 - sMAPE: 9.29%  |  MAE: 3.929
2015-01-10 - sMAPE: 11.49%  |  MAE: 4.531
2015-01-11 - sMAPE: 11.75%  |  MAE: 4.499
2015-01-12 - sMAPE: 11.70%  |  MAE: 4.481
2015-01-13 - sMAPE: 11.59%  |  MAE: 4.462
2015-01-14 - sMAPE: 11.33%  |  MAE: 4.377
2015-01-15 - sMAPE: 11.52%  |  MAE: 4.469
2015-01-16 - sMAPE: 11.16%  |  MAE: 4.321
2015-01-17 - sMAPE: 10.93%  |  MAE: 4.232
2015-01-18 - sMAPE: 11.13%  |  MAE: 4.293
2015-01-19 - sMAPE: 10.81%  |  MAE: 4.204
2015-01-20 - sMAPE: 10.50%  |  MAE: 4.106
2015-01-21 - sMAPE: 10.22%  |  MAE: 4.012
2015-01-22 - sMAPE: 9.98%  |  MAE: 3.945
2015-01-23 - sMAPE: 9.79%

2015-07-16 - sMAPE: 14.37%  |  MAE: 6.223
2015-07-17 - sMAPE: 14.33%  |  MAE: 6.208
2015-07-18 - sMAPE: 14.30%  |  MAE: 6.196
2015-07-19 - sMAPE: 14.30%  |  MAE: 6.190
2015-07-20 - sMAPE: 14.28%  |  MAE: 6.181
2015-07-21 - sMAPE: 14.28%  |  MAE: 6.183
2015-07-22 - sMAPE: 14.24%  |  MAE: 6.169
2015-07-23 - sMAPE: 14.21%  |  MAE: 6.158
2015-07-24 - sMAPE: 14.16%  |  MAE: 6.138
2015-07-25 - sMAPE: 14.26%  |  MAE: 6.159
2015-07-26 - sMAPE: 14.30%  |  MAE: 6.157
2015-07-27 - sMAPE: 14.35%  |  MAE: 6.164
2015-07-28 - sMAPE: 14.40%  |  MAE: 6.167
2015-07-29 - sMAPE: 14.39%  |  MAE: 6.155
2015-07-30 - sMAPE: 14.39%  |  MAE: 6.150
2015-07-31 - sMAPE: 14.40%  |  MAE: 6.149
2015-08-01 - sMAPE: 14.41%  |  MAE: 6.146
2015-08-02 - sMAPE: 14.44%  |  MAE: 6.152
2015-08-03 - sMAPE: 14.47%  |  MAE: 6.159
2015-08-04 - sMAPE: 14.45%  |  MAE: 6.149
2015-08-05 - sMAPE: 14.42%  |  MAE: 6.136
2015-08-06 - sMAPE: 14.41%  |  MAE: 6.128
2015-08-07 - sMAPE: 14.37%  |  MAE: 6.110
2015-08-08 - sMAPE: 14.39%  |  MAE

2016-01-28 - sMAPE: 15.41%  |  MAE: 7.227
2016-01-29 - sMAPE: 15.40%  |  MAE: 7.220
2016-01-30 - sMAPE: 15.40%  |  MAE: 7.211
2016-01-31 - sMAPE: 15.42%  |  MAE: 7.201
2016-02-01 - sMAPE: 15.46%  |  MAE: 7.203
2016-02-02 - sMAPE: 15.48%  |  MAE: 7.200
2016-02-03 - sMAPE: 15.50%  |  MAE: 7.197
2016-02-04 - sMAPE: 15.49%  |  MAE: 7.187
2016-02-05 - sMAPE: 15.51%  |  MAE: 7.182
2016-02-06 - sMAPE: 15.52%  |  MAE: 7.175
2016-02-07 - sMAPE: 15.58%  |  MAE: 7.173
2016-02-08 - sMAPE: 15.61%  |  MAE: 7.173
2016-02-09 - sMAPE: 15.60%  |  MAE: 7.163
2016-02-10 - sMAPE: 15.59%  |  MAE: 7.153
2016-02-11 - sMAPE: 15.57%  |  MAE: 7.141
2016-02-12 - sMAPE: 15.56%  |  MAE: 7.133
2016-02-13 - sMAPE: 15.58%  |  MAE: 7.127
2016-02-14 - sMAPE: 15.59%  |  MAE: 7.118
2016-02-15 - sMAPE: 15.63%  |  MAE: 7.126
2016-02-16 - sMAPE: 15.64%  |  MAE: 7.124
2016-02-17 - sMAPE: 15.63%  |  MAE: 7.118
2016-02-18 - sMAPE: 15.62%  |  MAE: 7.109
2016-02-19 - sMAPE: 15.62%  |  MAE: 7.103
2016-02-20 - sMAPE: 15.62%  |  MAE

2016-08-11 - sMAPE: 15.25%  |  MAE: 6.080
2016-08-12 - sMAPE: 15.24%  |  MAE: 6.074
2016-08-13 - sMAPE: 15.23%  |  MAE: 6.067
2016-08-14 - sMAPE: 15.24%  |  MAE: 6.064
2016-08-15 - sMAPE: 15.22%  |  MAE: 6.057
2016-08-16 - sMAPE: 15.22%  |  MAE: 6.055
2016-08-17 - sMAPE: 15.22%  |  MAE: 6.051
2016-08-18 - sMAPE: 15.20%  |  MAE: 6.044
2016-08-19 - sMAPE: 15.19%  |  MAE: 6.037
2016-08-20 - sMAPE: 15.18%  |  MAE: 6.030
2016-08-21 - sMAPE: 15.23%  |  MAE: 6.033
2016-08-22 - sMAPE: 15.22%  |  MAE: 6.028
2016-08-23 - sMAPE: 15.21%  |  MAE: 6.021
2016-08-24 - sMAPE: 15.19%  |  MAE: 6.016
2016-08-25 - sMAPE: 15.19%  |  MAE: 6.012
2016-08-26 - sMAPE: 15.17%  |  MAE: 6.005
2016-08-27 - sMAPE: 15.16%  |  MAE: 5.999
2016-08-28 - sMAPE: 15.16%  |  MAE: 5.994
2016-08-29 - sMAPE: 15.15%  |  MAE: 5.990
2016-08-30 - sMAPE: 15.15%  |  MAE: 5.988
2016-08-31 - sMAPE: 15.15%  |  MAE: 5.986
2016-09-01 - sMAPE: 15.13%  |  MAE: 5.980
2016-09-02 - sMAPE: 15.14%  |  MAE: 5.979
2016-09-03 - sMAPE: 15.14%  |  MAE

In [15]:
forecast

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,50.025486,44.851597,41.710289,35.305275,33.234028,33.564308,30.614616,28.661993,33.158356,40.951542,...,39.338554,37.707951,37.75489,46.356331,57.578003,57.336071,52.709045,46.330688,49.40263,48.2967
2015-01-02,44.643658,40.549091,36.881207,32.464886,31.821192,33.255219,37.205921,42.666275,44.663418,48.109207,...,44.347343,42.262154,42.911694,49.880184,56.483608,56.540211,52.654846,48.084156,50.78756,46.641079
2015-01-03,41.258575,36.143158,32.84206,28.639454,27.114294,29.109203,31.652597,34.951138,38.972366,44.047161,...,38.574173,36.875359,37.741051,45.364662,53.238808,53.644775,47.756981,41.63393,44.500595,41.959042
2015-01-04,36.851536,33.024899,30.623697,26.76079,25.085264,26.071133,28.301218,30.859612,34.342319,39.607098,...,39.708878,38.023735,39.417294,49.669949,54.677605,52.69854,49.290367,44.413322,44.79422,41.788136
2015-01-05,39.995358,36.81226,34.668224,30.498735,30.731709,34.363041,42.652508,57.453712,56.809467,56.312313,...,53.331543,51.7057,53.166904,63.367477,70.606331,62.762737,55.463032,49.438885,51.121063,48.305073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,35.787018,30.63962,29.369709,26.067051,25.688452,28.705256,38.086121,48.840275,50.660358,52.042606,...,45.799393,47.862209,49.226227,55.213245,58.393185,54.900551,51.358215,44.997898,46.363705,41.303963
2016-12-28,45.595814,42.150711,40.470146,36.957664,35.252975,38.611603,46.496521,56.793224,60.802917,59.852673,...,54.735279,54.272926,55.429058,65.357048,69.319542,63.144352,57.432987,50.983006,52.76321,49.740288
2016-12-29,41.512798,39.144749,37.45948,35.365021,35.057285,37.953651,46.985321,59.848145,62.455746,62.060043,...,54.148312,55.049732,56.72176,65.183937,70.480652,67.089256,59.263596,51.526272,52.060658,48.705414


In [16]:
forecast.to_csv("Resultados/dnn_BE_final.csv")

# DNN + Estandarización móvil

In [3]:
with open('dict_new_df_BE_epftoolbox_final.pkl', 'rb') as handle:
    dict_new_df = pickle.load(handle)

In [4]:
df_nuevo = pd.DataFrame(dict_new_df)

In [5]:
df_nuevo['Date'] = pd.to_datetime(df_nuevo.Date)

In [6]:
df_scalers = pd.DataFrame({'Date':dict_new_df['Date'], 'scaler':dict_new_df['scaler']})

In [7]:
df_scalers

,Date,scaler
0,2011-01-16 00:00:00,StandardScaler()
1,2011-01-16 01:00:00,StandardScaler()
2,2011-01-16 02:00:00,StandardScaler()
3,2011-01-16 03:00:00,StandardScaler()
4,2011-01-16 04:00:00,StandardScaler()
...,...,...
52243,2016-12-31 19:00:00,StandardScaler()
52244,2016-12-31 20:00:00,StandardScaler()
52245,2016-12-31 21:00:00,StandardScaler()
52246,2016-12-31 22:00:00,StandardScaler()


In [8]:
df_nuevo = df_nuevo[['Date', 'Price', 'Exogenous 1', 'Exogenous 2']]

In [9]:
df_nuevo = df_nuevo.set_index('Date')

In [10]:
df_nuevo.to_csv("Data/EPFToolbox/BE_dnn_estandarizacion_movil.csv")

In [11]:
from epftoolbox.models import hyperparameter_optimizer

In [12]:
# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = "BE_dnn_estandarizacion_movil"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2015 00:00"
end_test_date = "31/12/2016 23:00"

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = None

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 2

# Number of iterations for hyperparameter optimization
max_evals = 100

path_datasets_folder = "./Data/EPFToolbox/"
path_hyperparameters_folder = "./experimental_files/"

# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00


Tested 1/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 72.19 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.45 %


Tested 2/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 72.19 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.45 %


Tested 3/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 72.19 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.45 %


Tested 4/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 72.19 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.45 %


Tested 5/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 72.19 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.45 %


Tested 6/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 72.19 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.45 %


Tested 7/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 72.19 %

Best MAE - Test Dataset
  MAE



Tested 60/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.30 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 80.99 %


Tested 61/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.30 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 80.99 %


Tested 62/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.30 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 80.99 %


Tested 63/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.30 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 80.99 %


Tested 64/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.30 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 80.99 %


Tested 65/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.30 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 80.99 %


Tested 66/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.30 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 80.99 %


Tested 67/100 iterations.

In [13]:
"""
Example for using the DNN model for forecasting prices with daily recalibration
"""

# Author: Jesus Lago

# License: AGPL-3.0 License

import pandas as pd
import numpy as np
import argparse
import os

from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE
from epftoolbox.models import DNN


path_datasets_folder = "./Data/EPFToolbox/"
path_hyperparameters_folder = "./experimental_files/"
path_recalibration_folder = os.path.join('.', 'Resultados')
path_hyperparameter_folder = os.path.join('.', 'experimental_files')

dataset = "BE_dnn_estandarizacion_movil"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2015 00:00"
end_test_date = "31/12/2016 23:00"

# Defining train and testing data
df_train, df_test = read_data(dataset=dataset, years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

# Defining unique name to save the forecast
forecast_file_name = 'dnn_BE_estandarizacion_movil_final.csv'

forecast_file_path = os.path.join(path_recalibration_folder, forecast_file_name)

# # Defining empty forecast array and the real values to be predicted in a more friendly format
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

# # If we are not starting a new recalibration but re-starting an old one, we import the
# # existing files and print metrics 
# if not new_recalibration:
#     # Import existinf forecasting file
#     forecast = pd.read_csv(forecast_file_path, index_col=0)
#     forecast.index = pd.to_datetime(forecast.index)

#     # Reading dates to still be forecasted by checking NaN values
#     forecast_dates = forecast[forecast.isna().any(axis=1)].index

#     # If all the dates to be forecasted have already been forecast, we print information
#     # and exit the script
#     if len(forecast_dates) == 0:

#         mae = np.mean(MAE(forecast.values.squeeze(), real_values.values))
#         smape = np.mean(sMAPE(forecast.values.squeeze(), real_values.values)) * 100
#         print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format('Final metrics', smape, mae))
    
# else:
#     forecast_dates = forecast.index

forecast_dates = forecast.index

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = None
experiment_id = 2

model = DNN(
    experiment_id=experiment_id, path_hyperparameter_folder=path_hyperparameter_folder, nlayers=nlayers, 
    dataset=dataset, years_test=years_test, shuffle_train=shuffle_train, data_augmentation=data_augmentation,
    calibration_window=calibration_window)


# For loop over the recalibration dates
for date in forecast_dates:

    # For simulation purposes, we assume that the available data is
    # the data up to current date where the prices of current date are not known
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=23), :]], axis=0)

    # We extract real prices for current date and set them to NaN in the dataframe of available data
    data_available.loc[date:date + pd.Timedelta(hours=23), 'Price'] = np.NaN

    # Recalibrating the model with the most up-to-date available data and making a prediction
    # for the next day
    Yp = model.recalibrate_and_forecast_next_day(df=data_available, next_day_date=date)

    # Saving the current prediction
    forecast.loc[date, :] = Yp

    # Computing metrics up-to-current-date
    mae = np.mean(MAE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) 
    smape = np.mean(sMAPE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) * 100

    # Pringint information
    print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(str(date)[:10], smape, mae))

#     # Saving forecast
#     forecast.to_csv(forecast_file_path)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00
2015-01-01 - sMAPE: 66.90%  |  MAE: 0.356
2015-01-02 - sMAPE: 78.86%  |  MAE: 0.367
2015-01-03 - sMAPE: 85.95%  |  MAE: 0.347
2015-01-04 - sMAPE: 74.25%  |  MAE: 0.316
2015-01-05 - sMAPE: 70.53%  |  MAE: 0.310
2015-01-06 - sMAPE: 70.31%  |  MAE: 0.307
2015-01-07 - sMAPE: 66.96%  |  MAE: 0.299
2015-01-08 - sMAPE: 66.11%  |  MAE: 0.297
2015-01-09 - sMAPE: 71.61%  |  MAE: 0.339
2015-01-10 - sMAPE: 71.69%  |  MAE: 0.371
2015-01-11 - sMAPE: 68.98%  |  MAE: 0.368
2015-01-12 - sMAPE: 68.34%  |  MAE: 0.359
2015-01-13 - sMAPE: 68.00%  |  MAE: 0.364
2015-01-14 - sMAPE: 67.49%  |  MAE: 0.357
2015-01-15 - sMAPE: 68.42%  |  MAE: 0.376
2015-01-16 - sMAPE: 68.88%  |  MAE: 0.379
2015-01-17 - sMAPE: 70.84%  |  MAE: 0.376
2015-01-18 - sMAPE: 71.38%  |  MAE: 0.371
2015-01-19 - sMAPE: 68.83%  |  MAE: 0.362
2015-01-20 - sMAPE: 67.12%  |  MAE: 0.353
2015-01-21 - sMAPE: 66.54%  |  MAE: 0.350
2015-01-22 - sMAPE: 65.81%  |  MAE: 0.348
2015-01-23 - sMAPE:

2015-07-14 - sMAPE: 84.10%  |  MAE: 0.519
2015-07-15 - sMAPE: 84.09%  |  MAE: 0.519
2015-07-16 - sMAPE: 83.89%  |  MAE: 0.518
2015-07-17 - sMAPE: 83.78%  |  MAE: 0.517
2015-07-18 - sMAPE: 83.76%  |  MAE: 0.516
2015-07-19 - sMAPE: 83.81%  |  MAE: 0.516
2015-07-20 - sMAPE: 83.75%  |  MAE: 0.516
2015-07-21 - sMAPE: 83.49%  |  MAE: 0.515
2015-07-22 - sMAPE: 83.37%  |  MAE: 0.514
2015-07-23 - sMAPE: 83.15%  |  MAE: 0.513
2015-07-24 - sMAPE: 83.02%  |  MAE: 0.512
2015-07-25 - sMAPE: 83.15%  |  MAE: 0.515
2015-07-26 - sMAPE: 83.10%  |  MAE: 0.515
2015-07-27 - sMAPE: 83.25%  |  MAE: 0.516
2015-07-28 - sMAPE: 83.53%  |  MAE: 0.518
2015-07-29 - sMAPE: 83.80%  |  MAE: 0.518
2015-07-30 - sMAPE: 83.89%  |  MAE: 0.517
2015-07-31 - sMAPE: 83.74%  |  MAE: 0.517
2015-08-01 - sMAPE: 83.89%  |  MAE: 0.516
2015-08-02 - sMAPE: 83.87%  |  MAE: 0.518
2015-08-03 - sMAPE: 83.95%  |  MAE: 0.519
2015-08-04 - sMAPE: 83.87%  |  MAE: 0.519
2015-08-05 - sMAPE: 83.73%  |  MAE: 0.518
2015-08-06 - sMAPE: 83.54%  |  MAE

2016-01-26 - sMAPE: 84.83%  |  MAE: 0.552
2016-01-27 - sMAPE: 84.76%  |  MAE: 0.551
2016-01-28 - sMAPE: 84.70%  |  MAE: 0.550
2016-01-29 - sMAPE: 84.67%  |  MAE: 0.550
2016-01-30 - sMAPE: 84.59%  |  MAE: 0.549
2016-01-31 - sMAPE: 84.48%  |  MAE: 0.549
2016-02-01 - sMAPE: 84.44%  |  MAE: 0.549
2016-02-02 - sMAPE: 84.38%  |  MAE: 0.548
2016-02-03 - sMAPE: 84.34%  |  MAE: 0.548
2016-02-04 - sMAPE: 84.26%  |  MAE: 0.547
2016-02-05 - sMAPE: 84.30%  |  MAE: 0.547
2016-02-06 - sMAPE: 84.36%  |  MAE: 0.547
2016-02-07 - sMAPE: 84.26%  |  MAE: 0.546
2016-02-08 - sMAPE: 84.18%  |  MAE: 0.545
2016-02-09 - sMAPE: 84.06%  |  MAE: 0.544
2016-02-10 - sMAPE: 83.97%  |  MAE: 0.544
2016-02-11 - sMAPE: 83.84%  |  MAE: 0.543
2016-02-12 - sMAPE: 83.75%  |  MAE: 0.542
2016-02-13 - sMAPE: 83.73%  |  MAE: 0.541
2016-02-14 - sMAPE: 83.73%  |  MAE: 0.541
2016-02-15 - sMAPE: 83.68%  |  MAE: 0.541
2016-02-16 - sMAPE: 83.70%  |  MAE: 0.541
2016-02-17 - sMAPE: 83.68%  |  MAE: 0.541
2016-02-18 - sMAPE: 83.58%  |  MAE

2016-08-09 - sMAPE: 79.34%  |  MAE: 0.495
2016-08-10 - sMAPE: 79.30%  |  MAE: 0.494
2016-08-11 - sMAPE: 79.27%  |  MAE: 0.494
2016-08-12 - sMAPE: 79.24%  |  MAE: 0.494
2016-08-13 - sMAPE: 79.18%  |  MAE: 0.493
2016-08-14 - sMAPE: 79.16%  |  MAE: 0.493
2016-08-15 - sMAPE: 79.15%  |  MAE: 0.493
2016-08-16 - sMAPE: 79.08%  |  MAE: 0.492
2016-08-17 - sMAPE: 79.09%  |  MAE: 0.492
2016-08-18 - sMAPE: 79.04%  |  MAE: 0.492
2016-08-19 - sMAPE: 79.01%  |  MAE: 0.492
2016-08-20 - sMAPE: 78.97%  |  MAE: 0.492
2016-08-21 - sMAPE: 78.93%  |  MAE: 0.492
2016-08-22 - sMAPE: 78.96%  |  MAE: 0.493
2016-08-23 - sMAPE: 78.88%  |  MAE: 0.492
2016-08-24 - sMAPE: 78.82%  |  MAE: 0.492
2016-08-25 - sMAPE: 78.75%  |  MAE: 0.492
2016-08-26 - sMAPE: 78.69%  |  MAE: 0.491
2016-08-27 - sMAPE: 78.73%  |  MAE: 0.491
2016-08-28 - sMAPE: 78.70%  |  MAE: 0.491
2016-08-29 - sMAPE: 78.75%  |  MAE: 0.492
2016-08-30 - sMAPE: 78.76%  |  MAE: 0.492
2016-08-31 - sMAPE: 78.75%  |  MAE: 0.492
2016-09-01 - sMAPE: 78.67%  |  MAE

In [14]:
forecast

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,0.206741,-0.211487,-0.551184,-0.966076,-1.157379,-1.084329,-1.142526,-1.165197,-0.953388,-0.513042,...,-0.48191,-0.615783,-0.55074,0.155915,0.745975,0.726184,0.528555,0.250687,0.331014,0.149125
2015-01-02,-0.088213,-0.320958,-0.561313,-0.885501,-1.033667,-0.806056,-0.548037,-0.158525,-0.062525,0.166987,...,0.077462,-0.049757,-0.043171,0.623624,0.999286,0.815313,0.441568,0.137775,0.411041,0.105482
2015-01-03,-0.134765,-0.562445,-0.943951,-1.23867,-1.416443,-1.280484,-0.941804,-0.677915,-0.372419,0.142353,...,-0.377499,-0.510093,-0.554915,0.216934,0.722248,0.649231,0.255523,-0.213146,0.156061,-0.115322
2015-01-04,-0.722765,-1.260163,-1.534508,-1.958582,-2.116531,-2.027969,-1.982139,-2.083417,-1.648052,-1.150412,...,-0.945547,-1.066819,-0.970608,-0.145209,0.570311,0.774188,0.519006,0.126786,0.11128,-0.314731
2015-01-05,-0.645726,-0.96426,-1.130483,-1.457179,-1.504862,-1.151965,-0.37025,0.560397,0.620938,0.738467,...,0.557941,0.532976,0.628917,1.701046,2.07934,1.46867,0.723375,0.163618,0.35285,0.112869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,-0.548948,-0.766949,-0.980699,-1.248172,-1.338172,-1.049881,-0.469115,0.27341,0.305815,0.373187,...,0.029476,-0.004227,0.071341,0.672945,0.836965,0.559457,0.071781,-0.299934,-0.114878,-0.231325
2016-12-28,0.077766,-0.117934,-0.307012,-0.553492,-0.655394,-0.411547,-0.048751,0.386666,0.746571,0.711263,...,0.344342,0.260252,0.333746,1.033218,1.507118,0.953278,0.332146,-0.124463,0.161474,0.052527
2016-12-29,-0.280932,-0.436015,-0.565711,-0.694904,-0.741232,-0.591561,-0.037488,0.831967,0.945465,0.96191,...,0.530149,0.47829,0.601117,1.410077,2.012687,1.536412,0.978815,0.412246,0.440493,0.214346


In [15]:
forecast.to_csv("Resultados/dnn_BE_estandarizacion_movil_final.csv")

In [16]:
df_test_original = pd.DataFrame(forecast.values.reshape((-1,)), columns=['Price'])
df_test_original['Date'] = df_scalers.tail(len(df_test_original)).Date.to_numpy()

In [17]:
predicciones = np.array([])
for cont in range(0, len(df_test_original)):
    
    print(df_test_original.iloc[cont].Date)
    
    # Acumulamos las predicciones
    scaler_obj = df_scalers[df_scalers.Date == df_test_original.iloc[cont].Date].scaler.values[0]

    predicciones = np.append(predicciones, scaler_obj.inverse_transform(np.array(df_test_original.iloc[cont].Price).reshape(1, -1))[0][0])

2015-01-01 00:00:00
2015-01-01 01:00:00
2015-01-01 02:00:00
2015-01-01 03:00:00
2015-01-01 04:00:00
2015-01-01 05:00:00
2015-01-01 06:00:00
2015-01-01 07:00:00
2015-01-01 08:00:00
2015-01-01 09:00:00
2015-01-01 10:00:00
2015-01-01 11:00:00
2015-01-01 12:00:00
2015-01-01 13:00:00
2015-01-01 14:00:00
2015-01-01 15:00:00
2015-01-01 16:00:00
2015-01-01 17:00:00
2015-01-01 18:00:00
2015-01-01 19:00:00
2015-01-01 20:00:00
2015-01-01 21:00:00
2015-01-01 22:00:00
2015-01-01 23:00:00
2015-01-02 00:00:00
2015-01-02 01:00:00
2015-01-02 02:00:00
2015-01-02 03:00:00
2015-01-02 04:00:00
2015-01-02 05:00:00
2015-01-02 06:00:00
2015-01-02 07:00:00
2015-01-02 08:00:00
2015-01-02 09:00:00
2015-01-02 10:00:00
2015-01-02 11:00:00
2015-01-02 12:00:00
2015-01-02 13:00:00
2015-01-02 14:00:00
2015-01-02 15:00:00
2015-01-02 16:00:00
2015-01-02 17:00:00
2015-01-02 18:00:00
2015-01-02 19:00:00
2015-01-02 20:00:00
2015-01-02 21:00:00
2015-01-02 22:00:00
2015-01-02 23:00:00
2015-01-03 00:00:00
2015-01-03 01:00:00


2015-01-20 02:00:00
2015-01-20 03:00:00
2015-01-20 04:00:00
2015-01-20 05:00:00
2015-01-20 06:00:00
2015-01-20 07:00:00
2015-01-20 08:00:00
2015-01-20 09:00:00
2015-01-20 10:00:00
2015-01-20 11:00:00
2015-01-20 12:00:00
2015-01-20 13:00:00
2015-01-20 14:00:00
2015-01-20 15:00:00
2015-01-20 16:00:00
2015-01-20 17:00:00
2015-01-20 18:00:00
2015-01-20 19:00:00
2015-01-20 20:00:00
2015-01-20 21:00:00
2015-01-20 22:00:00
2015-01-20 23:00:00
2015-01-21 00:00:00
2015-01-21 01:00:00
2015-01-21 02:00:00
2015-01-21 03:00:00
2015-01-21 04:00:00
2015-01-21 05:00:00
2015-01-21 06:00:00
2015-01-21 07:00:00
2015-01-21 08:00:00
2015-01-21 09:00:00
2015-01-21 10:00:00
2015-01-21 11:00:00
2015-01-21 12:00:00
2015-01-21 13:00:00
2015-01-21 14:00:00
2015-01-21 15:00:00
2015-01-21 16:00:00
2015-01-21 17:00:00
2015-01-21 18:00:00
2015-01-21 19:00:00
2015-01-21 20:00:00
2015-01-21 21:00:00
2015-01-21 22:00:00
2015-01-21 23:00:00
2015-01-22 00:00:00
2015-01-22 01:00:00
2015-01-22 02:00:00
2015-01-22 03:00:00


2015-02-06 07:00:00
2015-02-06 08:00:00
2015-02-06 09:00:00
2015-02-06 10:00:00
2015-02-06 11:00:00
2015-02-06 12:00:00
2015-02-06 13:00:00
2015-02-06 14:00:00
2015-02-06 15:00:00
2015-02-06 16:00:00
2015-02-06 17:00:00
2015-02-06 18:00:00
2015-02-06 19:00:00
2015-02-06 20:00:00
2015-02-06 21:00:00
2015-02-06 22:00:00
2015-02-06 23:00:00
2015-02-07 00:00:00
2015-02-07 01:00:00
2015-02-07 02:00:00
2015-02-07 03:00:00
2015-02-07 04:00:00
2015-02-07 05:00:00
2015-02-07 06:00:00
2015-02-07 07:00:00
2015-02-07 08:00:00
2015-02-07 09:00:00
2015-02-07 10:00:00
2015-02-07 11:00:00
2015-02-07 12:00:00
2015-02-07 13:00:00
2015-02-07 14:00:00
2015-02-07 15:00:00
2015-02-07 16:00:00
2015-02-07 17:00:00
2015-02-07 18:00:00
2015-02-07 19:00:00
2015-02-07 20:00:00
2015-02-07 21:00:00
2015-02-07 22:00:00
2015-02-07 23:00:00
2015-02-08 00:00:00
2015-02-08 01:00:00
2015-02-08 02:00:00
2015-02-08 03:00:00
2015-02-08 04:00:00
2015-02-08 05:00:00
2015-02-08 06:00:00
2015-02-08 07:00:00
2015-02-08 08:00:00


2015-02-23 22:00:00
2015-02-23 23:00:00
2015-02-24 00:00:00
2015-02-24 01:00:00
2015-02-24 02:00:00
2015-02-24 03:00:00
2015-02-24 04:00:00
2015-02-24 05:00:00
2015-02-24 06:00:00
2015-02-24 07:00:00
2015-02-24 08:00:00
2015-02-24 09:00:00
2015-02-24 10:00:00
2015-02-24 11:00:00
2015-02-24 12:00:00
2015-02-24 13:00:00
2015-02-24 14:00:00
2015-02-24 15:00:00
2015-02-24 16:00:00
2015-02-24 17:00:00
2015-02-24 18:00:00
2015-02-24 19:00:00
2015-02-24 20:00:00
2015-02-24 21:00:00
2015-02-24 22:00:00
2015-02-24 23:00:00
2015-02-25 00:00:00
2015-02-25 01:00:00
2015-02-25 02:00:00
2015-02-25 03:00:00
2015-02-25 04:00:00
2015-02-25 05:00:00
2015-02-25 06:00:00
2015-02-25 07:00:00
2015-02-25 08:00:00
2015-02-25 09:00:00
2015-02-25 10:00:00
2015-02-25 11:00:00
2015-02-25 12:00:00
2015-02-25 13:00:00
2015-02-25 14:00:00
2015-02-25 15:00:00
2015-02-25 16:00:00
2015-02-25 17:00:00
2015-02-25 18:00:00
2015-02-25 19:00:00
2015-02-25 20:00:00
2015-02-25 21:00:00
2015-02-25 22:00:00
2015-02-25 23:00:00


2015-03-13 14:00:00
2015-03-13 15:00:00
2015-03-13 16:00:00
2015-03-13 17:00:00
2015-03-13 18:00:00
2015-03-13 19:00:00
2015-03-13 20:00:00
2015-03-13 21:00:00
2015-03-13 22:00:00
2015-03-13 23:00:00
2015-03-14 00:00:00
2015-03-14 01:00:00
2015-03-14 02:00:00
2015-03-14 03:00:00
2015-03-14 04:00:00
2015-03-14 05:00:00
2015-03-14 06:00:00
2015-03-14 07:00:00
2015-03-14 08:00:00
2015-03-14 09:00:00
2015-03-14 10:00:00
2015-03-14 11:00:00
2015-03-14 12:00:00
2015-03-14 13:00:00
2015-03-14 14:00:00
2015-03-14 15:00:00
2015-03-14 16:00:00
2015-03-14 17:00:00
2015-03-14 18:00:00
2015-03-14 19:00:00
2015-03-14 20:00:00
2015-03-14 21:00:00
2015-03-14 22:00:00
2015-03-14 23:00:00
2015-03-15 00:00:00
2015-03-15 01:00:00
2015-03-15 02:00:00
2015-03-15 03:00:00
2015-03-15 04:00:00
2015-03-15 05:00:00
2015-03-15 06:00:00
2015-03-15 07:00:00
2015-03-15 08:00:00
2015-03-15 09:00:00
2015-03-15 10:00:00
2015-03-15 11:00:00
2015-03-15 12:00:00
2015-03-15 13:00:00
2015-03-15 14:00:00
2015-03-15 15:00:00


...

2016-10-12 19:00:00
2016-10-12 20:00:00
2016-10-12 21:00:00
2016-10-12 22:00:00
2016-10-12 23:00:00
2016-10-13 00:00:00
2016-10-13 01:00:00
2016-10-13 02:00:00
2016-10-13 03:00:00
2016-10-13 04:00:00
2016-10-13 05:00:00
2016-10-13 06:00:00
2016-10-13 07:00:00
2016-10-13 08:00:00
2016-10-13 09:00:00
2016-10-13 10:00:00
2016-10-13 11:00:00
2016-10-13 12:00:00
2016-10-13 13:00:00
2016-10-13 14:00:00
2016-10-13 15:00:00
2016-10-13 16:00:00
2016-10-13 17:00:00
2016-10-13 18:00:00
2016-10-13 19:00:00
2016-10-13 20:00:00
2016-10-13 21:00:00
2016-10-13 22:00:00
2016-10-13 23:00:00
2016-10-14 00:00:00
2016-10-14 01:00:00
2016-10-14 02:00:00
2016-10-14 03:00:00
2016-10-14 04:00:00
2016-10-14 05:00:00
2016-10-14 06:00:00
2016-10-14 07:00:00
2016-10-14 08:00:00
2016-10-14 09:00:00
2016-10-14 10:00:00
2016-10-14 11:00:00
2016-10-14 12:00:00
2016-10-14 13:00:00
2016-10-14 14:00:00
2016-10-14 15:00:00
2016-10-14 16:00:00
2016-10-14 17:00:00
2016-10-14 18:00:00
2016-10-14 19:00:00
2016-10-14 20:00:00


2016-10-31 15:00:00
2016-10-31 16:00:00
2016-10-31 17:00:00
2016-10-31 18:00:00
2016-10-31 19:00:00
2016-10-31 20:00:00
2016-10-31 21:00:00
2016-10-31 22:00:00
2016-10-31 23:00:00
2016-11-01 00:00:00
2016-11-01 01:00:00
2016-11-01 02:00:00
2016-11-01 03:00:00
2016-11-01 04:00:00
2016-11-01 05:00:00
2016-11-01 06:00:00
2016-11-01 07:00:00
2016-11-01 08:00:00
2016-11-01 09:00:00
2016-11-01 10:00:00
2016-11-01 11:00:00
2016-11-01 12:00:00
2016-11-01 13:00:00
2016-11-01 14:00:00
2016-11-01 15:00:00
2016-11-01 16:00:00
2016-11-01 17:00:00
2016-11-01 18:00:00
2016-11-01 19:00:00
2016-11-01 20:00:00
2016-11-01 21:00:00
2016-11-01 22:00:00
2016-11-01 23:00:00
2016-11-02 00:00:00
2016-11-02 01:00:00
2016-11-02 02:00:00
2016-11-02 03:00:00
2016-11-02 04:00:00
2016-11-02 05:00:00
2016-11-02 06:00:00
2016-11-02 07:00:00
2016-11-02 08:00:00
2016-11-02 09:00:00
2016-11-02 10:00:00
2016-11-02 11:00:00
2016-11-02 12:00:00
2016-11-02 13:00:00
2016-11-02 14:00:00
2016-11-02 15:00:00
2016-11-02 16:00:00


2016-11-19 14:00:00
2016-11-19 15:00:00
2016-11-19 16:00:00
2016-11-19 17:00:00
2016-11-19 18:00:00
2016-11-19 19:00:00
2016-11-19 20:00:00
2016-11-19 21:00:00
2016-11-19 22:00:00
2016-11-19 23:00:00
2016-11-20 00:00:00
2016-11-20 01:00:00
2016-11-20 02:00:00
2016-11-20 03:00:00
2016-11-20 04:00:00
2016-11-20 05:00:00
2016-11-20 06:00:00
2016-11-20 07:00:00
2016-11-20 08:00:00
2016-11-20 09:00:00
2016-11-20 10:00:00
2016-11-20 11:00:00
2016-11-20 12:00:00
2016-11-20 13:00:00
2016-11-20 14:00:00
2016-11-20 15:00:00
2016-11-20 16:00:00
2016-11-20 17:00:00
2016-11-20 18:00:00
2016-11-20 19:00:00
2016-11-20 20:00:00
2016-11-20 21:00:00
2016-11-20 22:00:00
2016-11-20 23:00:00
2016-11-21 00:00:00
2016-11-21 01:00:00
2016-11-21 02:00:00
2016-11-21 03:00:00
2016-11-21 04:00:00
2016-11-21 05:00:00
2016-11-21 06:00:00
2016-11-21 07:00:00
2016-11-21 08:00:00
2016-11-21 09:00:00
2016-11-21 10:00:00
2016-11-21 11:00:00
2016-11-21 12:00:00
2016-11-21 13:00:00
2016-11-21 14:00:00
2016-11-21 15:00:00


2016-12-07 15:00:00
2016-12-07 16:00:00
2016-12-07 17:00:00
2016-12-07 18:00:00
2016-12-07 19:00:00
2016-12-07 20:00:00
2016-12-07 21:00:00
2016-12-07 22:00:00
2016-12-07 23:00:00
2016-12-08 00:00:00
2016-12-08 01:00:00
2016-12-08 02:00:00
2016-12-08 03:00:00
2016-12-08 04:00:00
2016-12-08 05:00:00
2016-12-08 06:00:00
2016-12-08 07:00:00
2016-12-08 08:00:00
2016-12-08 09:00:00
2016-12-08 10:00:00
2016-12-08 11:00:00
2016-12-08 12:00:00
2016-12-08 13:00:00
2016-12-08 14:00:00
2016-12-08 15:00:00
2016-12-08 16:00:00
2016-12-08 17:00:00
2016-12-08 18:00:00
2016-12-08 19:00:00
2016-12-08 20:00:00
2016-12-08 21:00:00
2016-12-08 22:00:00
2016-12-08 23:00:00
2016-12-09 00:00:00
2016-12-09 01:00:00
2016-12-09 02:00:00
2016-12-09 03:00:00
2016-12-09 04:00:00
2016-12-09 05:00:00
2016-12-09 06:00:00
2016-12-09 07:00:00
2016-12-09 08:00:00
2016-12-09 09:00:00
2016-12-09 10:00:00
2016-12-09 11:00:00
2016-12-09 12:00:00
2016-12-09 13:00:00
2016-12-09 14:00:00
2016-12-09 15:00:00
2016-12-09 16:00:00


2016-12-25 11:00:00
2016-12-25 12:00:00
2016-12-25 13:00:00
2016-12-25 14:00:00
2016-12-25 15:00:00
2016-12-25 16:00:00
2016-12-25 17:00:00
2016-12-25 18:00:00
2016-12-25 19:00:00
2016-12-25 20:00:00
2016-12-25 21:00:00
2016-12-25 22:00:00
2016-12-25 23:00:00
2016-12-26 00:00:00
2016-12-26 01:00:00
2016-12-26 02:00:00
2016-12-26 03:00:00
2016-12-26 04:00:00
2016-12-26 05:00:00
2016-12-26 06:00:00
2016-12-26 07:00:00
2016-12-26 08:00:00
2016-12-26 09:00:00
2016-12-26 10:00:00
2016-12-26 11:00:00
2016-12-26 12:00:00
2016-12-26 13:00:00
2016-12-26 14:00:00
2016-12-26 15:00:00
2016-12-26 16:00:00
2016-12-26 17:00:00
2016-12-26 18:00:00
2016-12-26 19:00:00
2016-12-26 20:00:00
2016-12-26 21:00:00
2016-12-26 22:00:00
2016-12-26 23:00:00
2016-12-27 00:00:00
2016-12-27 01:00:00
2016-12-27 02:00:00
2016-12-27 03:00:00
2016-12-27 04:00:00
2016-12-27 05:00:00
2016-12-27 06:00:00
2016-12-27 07:00:00
2016-12-27 08:00:00
2016-12-27 09:00:00
2016-12-27 10:00:00
2016-12-27 11:00:00
2016-12-27 12:00:00


In [18]:
df_test_original['Price'] = predicciones

In [19]:
df_test_original = df_test_original.set_index(df_test_original.Date).drop('Date', axis = 1)

In [20]:
forecast_final = df_test_original.loc[:, ['Price']].values.reshape(-1, 24)
forecast_final = pd.DataFrame(forecast_final, index=df_test_original.index[::24], columns=['h' + str(k) for k in range(24)])

In [21]:
forecast_final

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,45.600097,41.046682,37.348274,32.831193,30.748403,31.543729,30.910109,30.663288,32.969329,37.763544,...,38.102494,36.644967,37.353106,45.046730,51.470943,51.255462,49.103809,46.078556,46.953102,44.972809
2015-01-02,43.147982,40.753137,38.279986,34.944233,33.419661,35.761688,38.416588,42.424500,43.412301,45.773879,...,44.852701,43.543677,43.611448,50.472481,54.337878,52.444877,48.599197,45.473299,48.285095,45.141015
2015-01-03,42.474800,38.106207,34.209275,31.198835,29.382952,30.771723,34.231205,36.926732,40.047258,45.305448,...,39.995362,38.640966,38.183124,46.067270,51.228856,50.483010,46.461433,41.674167,45.445477,42.673400
2015-01-04,37.714876,32.967294,30.543622,26.797197,25.401815,26.184199,26.589081,25.694354,29.540535,33.936879,...,35.746733,34.675373,35.525339,42.817224,49.138407,50.939535,48.685155,45.220132,45.083154,41.319601
2015-01-05,38.245320,35.620818,34.251260,31.559514,31.166634,34.074261,40.515048,48.182941,48.681755,49.650111,...,48.162704,47.957009,48.747493,57.581096,60.697978,55.666474,49.525760,44.913754,46.472890,44.495615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,40.268741,36.680470,33.162168,28.759606,27.278212,32.023453,41.582779,53.804632,54.338018,55.446957,...,49.789516,49.234762,50.478615,60.380924,63.080667,58.512933,50.485857,44.367467,47.413474,45.496766
2016-12-28,48.038198,45.091209,42.243954,38.532298,36.997781,40.669791,46.133021,52.689821,58.109510,57.577806,...,52.052466,50.786184,51.892905,62.426016,69.562320,61.222240,51.868824,44.992900,49.298720,47.658135
2016-12-29,41.987466,39.671300,37.734308,35.804817,35.112917,37.348236,45.623277,58.608516,60.303593,60.549196,...,54.100883,53.326383,55.160780,67.242539,76.242472,69.129340,60.801672,52.340008,52.761880,49.384399


In [22]:
forecast_final.to_csv("Resultados/dnn_BE_estandarizacion_movil_final.csv")

In [23]:
_, df_test = read_data(dataset="BE_dnn", years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00


In [24]:
mae = np.mean(MAE(forecast_final.values.squeeze(), real_values.values)) 
smape = np.mean(sMAPE(forecast_final.values.squeeze(), real_values.values)) * 100
print('sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(smape, mae))

sMAPE: 16.22%  |  MAE: 7.209
